Imports

In [ ]:
import sys 
import numpy as np
import matplotlib.pyplot as plt 
import random 
import scipy 
import PIL
import cv2 
import time
import imageio
import scipy.ndimage
import torch 
import torch.nn as nn
from torch import optim
from matplotlib.pyplot import colormaps as cm 
from mpl_toolkits.mplot3d import Axes3D
from scipy import misc as ndimage
from skfmm import distance 
from os import listdir 
from PIL import Image, ImageMath 
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

model.py (pytorch)

In [ ]:
class Conv_Model(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(Conv_Model, self).__init__()
    self.in_channels = in_channels
    self.out = out_channels
    self.conv1 = nn.Conv2d(self.in_channels, self.out, kernel_size=3, stride = 1, padding=2, dilation=2) 
    self.conv2 = nn.Conv2d(self.out, self.out, kernel_size=3, stride = 1, padding=2, dilation=2)
    self.bn1 = nn.BatchNorm2d(self.out)
    self.bn2 = nn.BatchNorm2d(self.out)
    self.relu = nn.LeakyReLU(0.33)
    
  def forward(self, x, bn_use):
    if bn_use == True:
      output = self.conv1(x)
      output = self.bn1(output)
      output = self.relu(output)
      output = self.conv2(output)
      output = self.bn2(output)
      output = self.relu(output)
    else:
      output = self.conv1(x)
      output = self.relu(output)
      output = self.conv2(output)
      output = self.relu(output)
    return output


class Down_Sample(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(Down_Sample, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.conv = Conv_Model(self.in_channels, self.out_channels)
    self.maxpool = nn.MaxPool2d(2)
    
  def forward(self, x, bn_use):
    output = self.maxpool(x)
    output = self.conv(output, bn_use)
    return output

class Up_Sample(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(Up_Sample, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.up_1 = nn.ConvTranspose2d(self.in_channels, self.out_channels, kernel_size=2, stride=2) 
    self.up_model = Conv_Model(self.in_channels, self.out_channels)

  def forward(self, x1, x2, bn_use):
    x1 = self.up_1(x1)
    x = torch.cat([x2, x1], dim=1)
    return self.up_model(x, bn_use)


class model(nn.Module):
  def __init__(self, channels, device = torch.device("cuda")):
    super(model, self).__init__()
    self.in_channels = channels
    self.stride = 1
    self.out = 4
    self.device = device

    self.conv_model = Conv_Model(self.in_channels, self.out)
    self.down1 = Down_Sample(self.out, 2*self.out)
    self.down2 = Down_Sample(2*self.out, 4*self.out)
    self.down3 = Down_Sample(4*self.out, 8*self.out)
    self.down4 = Down_Sample(8*self.out, 16*self.out)
    self.down5 = Down_Sample(16*self.out, 32*self.out)
    self.down6 = Down_Sample(32*self.out, 64*self.out)
    self.up5 = Up_Sample(64*self.out, 32*self.out)
    self.up4 = Up_Sample(32*self.out, 16*self.out)
    self.up3 = Up_Sample(16*self.out, 8*self.out)
    self.up2 = Up_Sample(8*self.out, 4*self.out)
    self.up1 = Up_Sample(4*self.out, 2*self.out)
    self.up0 = Up_Sample(2*self.out, self.out)
    self.output = nn.Conv2d(self.out, 1, kernel_size = 1)

  def forward(self, x):
    x1 = self.conv_model(x, True) 
    x1_down = self.down1(x1, True) 
    x2_down = self.down2(x1_down, True) 
    x3_down = self.down3(x2_down, True) 
    x4_down = self.down4(x3_down, True) 
    x5_down = self.down5(x4_down, False)
    x6_down = self.down6(x5_down, False) 
    x5_up = self.up5(x6_down, x5_down, True)
    x4_up = self.up4(x5_up, x4_down, True)
    x3_up = self.up3(x4_up, x3_down, True) 
    x2_up = self.up2(x3_up, x2_down, True) 
    x1_up = self.up1(x2_up, x1_down, True) 
    x0_up = self.up0(x1_up, x1, True) 
    logits = self.output(x0_up)
    sig = nn.Sigmoid()
    output = sig(logits)
    return output

train.py (pytorch)

In [ ]:
def train(train_input, train_label, device=torch.device("cuda")):
  base_dir = '/content/drive/MyDrive/Colab_Notebooks'
  train_label.requires_grad=False

  # 1 channel for frontiers and 1 for cumulative visibility
  train_model  = model(channels=2, device = device).double().to(device) 
  optimizer = optim.Adam(train_model.parameters(), lr = 1e-3)

  # Schedule decrease in learning rate every 200 epochs
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones = [200,400,600,800,1000], gamma = 0.1)

  loss_fn = nn.BCELoss()
  loss_fn_eval = nn.BCELoss()
  loss_list = []
  loss_list_eval = []
  num_epochs = 10000
  batchsize = 25
  data = TensorDataset(train_input, train_label)
  full_size = len(train_label)
  train_size = int(0.9*full_size)
  test_size = full_size - train_size
  train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
  train_loader = DataLoader(dataset=train_dataset, batch_size = batchsize, shuffle = True)
  test_loader = DataLoader(dataset=test_dataset, batch_size=batchsize, shuffle=False)

  # Feed through all the target data
  for epoch in range(num_epochs):
    running_loss = 0.0
    running_loss_eval = 0.0
    counter = 0
    train_model.train() # Put model in training mode
    start = time.time()
    for xbatch, ybatch in train_loader:
      xbatch = xbatch.to(device)
      ybatch = ybatch.to(device)
      optimizer.zero_grad()
      output = train_model(xbatch).squeeze()
      loss = loss_fn(output, ybatch)
      loss.backward(retain_graph=True)
      
      #if (epoch < 25 or epoch > 0.5*num_epochs) and counter%999==0:
      if counter%999==0:
        print('expected training gain is:')
        plt.imshow(ybatch[0].cpu().detach().numpy()); plt.show(); plt.close()
        print('output training gain is:')
        plt.imshow(output[0].cpu().detach().numpy()); plt.show(); plt.close()
        print('LOSS IS:', loss)
        print('Iteration {} completed'.format(counter+1))
        print()

      optimizer.step()
      scheduler.step()
      counter+=1
      running_loss += loss.item()
      torch.cuda.empty_cache
      counter += 1
    
    # Validate model after each epoch
    train_model.eval()
    with torch.no_grad():
      for xbatch_eval, ybatch_eval in test_loader:
        xbatch_eval = xbatch_eval.to(device)
        ybatch_eval = ybatch_eval.to(device)
        output_eval = train_model(xbatch_eval).squeeze()
        loss_eval = loss_fn_eval(output_eval, ybatch_eval)
        running_loss_eval += loss_eval.item()
        torch.cuda.empty_cache
      
    end = time.time()
    total_time = end-start
    minutes = total_time//60
    seconds = total_time%60
    running_loss = running_loss/train_size
    running_loss_eval = running_loss_eval/test_size
    loss_list.append(running_loss)
    loss_list_eval.append(running_loss_eval)
    
    print('Training loss is')
    plt.plot(loss_list); plt.show(); plt.close()
    print('Validation loss is')
    plt.plot(loss_list_eval); plt.show(); plt.close()
    
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('Running training loss is:', running_loss)
    print('Running validation loss is:', running_loss_eval)
    
    if (epoch+1)%10==0:
      torch.save(train_model.state_dict(), base_dir + "/Models3/bce_BN_model_{}.pt".format(epoch+1))
      torch.save(loss_list, base_dir + "/Losses3/bce_BN_loss_{}.pt".format(epoch+1))
      torch.save(loss_list_eval, base_dir + "/Losses3/bce_BN_validation_loss_{}.pt".format(epoch+1))
    print('Time is:', minutes,'minutes and', seconds, 'seconds')
    print()

  return train_model, loss_list

Main

In [ ]:
# Load the data

train_data= load_data()
train_imgs, train_labels =  zip(*train_data)
train_labels = list(train_labels)

states = torch.stack(train_imgs)
target = torch.stack(train_labels)

In [ ]:
def main():
  device=torch.device("cuda")
  base_dir = '/content/drive/MyDrive/Colab_Notebooks'

  model, loss = train(states, target, device = device)
  torch.save(model.state_dict(), base_dir + "/model_bce_final.pt")
  
if __name__ == '__main__':
    main()

Load Data

In [ ]:
import torch
device=torch.device("cuda")

def load_data():
  base_dir = '/content/drive/MyDrive/Colab_Notebooks'
  train_data = torch.load(base_dir + '/train_data_filtered.pt')
  return train_data

Generate Training Examples

In [ ]:
def generate_training():
  # Get cumulative visibility and frontiers for input

  #seed_everything() # Seed to get reproducible results
  for i in range(1):
    print('current iteration is:', i)
    run_sequence(i)

generate_training()

Greedy Algorithm Functions

In [ ]:
def vis2d(phi, O):
# phi is the sdf of the scene
# O is the grid index of the vantage point

    psi = 1*phi

    # streamlined sweeping so we don't have to think about the indices
    for i in range(2):
        si = i*1 + i-1   # gives {-1 or 1}
        for j in range(2):
            sj = j*1 + j-1
            psi = sweep2d(phi, psi, O, si, sj)

    return psi


def compute_max_index(N, s, O):
# determine how many steps to take based on if sweeping forward or backwards
    if s == 1:
        indMax = N-O
    elif s == -1:
        indMax = O -(-1)
    return indMax


def sweep2d(phi, psi, O, si, sj):
# a way to write the sweep so that it is modular
# basically propagates information from the origin.
# si,sj,sk determine the direction of the indices and the sign of r

# need to turn off boundscheck for faster

    Ny = phi.shape[0]
    Nx = phi.shape[1]

    # compute indices
    I = compute_max_index(Ny, si, O[0])
    J = compute_max_index(Nx, sj, O[1])

    # a weird way to index the loop so that it can use c compatible range function
    for ri in range(I):
        i = O[0] + si*ri
        for rj in range(J):
            j = O[1] + sj*rj
            if (ri+rj !=0 ):  # this if should be removed but doesn't seem to affect much
                A = 1.0/(ri+rj)

                # clamp indices between 0 and N. The edges shouldn't matter because ri, rj would be zero if they fall off
                i_ = max(0, min(Ny-1, (i-si) ) )
                j_ = max(0, min(Nx-1, (j-sj) ) )
                psi[i,j] = A*(ri*psi[i_,j]+rj*psi[i,j_])
                psi[i,j] = min(phi[i,j],psi[i,j])

    return psi

def delta(x, epsilon):
    chi = (x>(-epsilon/2)) * (x<(epsilon/2))
    y = 2/epsilon * chi * np.cos(np.pi * x / epsilon) **2
    return y

def im2double(img): # Normalize array
    min_val = np.min(1*img.ravel())
    max_val = np.max(1*img.ravel())
    out = (img.astype('float') - min_val) / (max_val - min_val)
    return out

def plot_pos(phi, x0_one = None, x0_two = None, h_1 = None, x0 = None):
  # Plot positive part (usually for cumulative visibility and frontiers)
	plt.imshow(phi>0, cmap = 'gray')
	if h_1 != None:
		plt.plot(x0_one, x0_two, 'ro') 
		for i in range (1,h_1):
			plt.plot((x0[i])[1], (x0[i])[0], 'bo') #; plt.plot(x1[1], x1[0], 'ro'); 		
	if x0_one != None and x0_two != None:
		plt.plot(x0_one, x0_two, 'ro')
	plt.show(); plt.close()

def initialize_point(img, dx): 
	# Initialize point in free space--so where ever there is white space (255) or 1 when normalized
	free_space = False
	while free_space == False:
		x_pos = random.randint(0, img.shape[0]-1)
		y_pos = random.randint(0, img.shape[1]-1)
	return [x_pos*dx, y_pos*dx]

In [ ]:
def seed_everything(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

Greedy Algorithm

In [ ]:
import numpy.random as random
from skfmm import distance

def setup(h, w):
  # SET UP A GRID OVER [-1,1]x[-1,1]
  m = h
  dx = 2/m
  grid_space = np.linspace(-1,1-dx,m)
  [x,y] = np.meshgrid(grid_space, grid_space) # 2D Array (list)

  # CREATE LEVEL SET FUNCTION
  num_circles = random.randint(2,6)
  phi = np.ndarray(shape=(h,w), dtype=float)
  x_vals = [] # x_vals and y_vals are the centers of the circles
  y_vals = []
  radii = []
  for i in range(num_circles):
    x_val = random.uniform(-0.8,0.8)
    y_val = random.uniform(-0.8,0.8)
    rad_val = random.uniform(0.12,0.46) 
    x_vals.append(x_val)
    y_vals.append(y_val)
    radii.append(rad_val)

  norm_factor = 0
  for i in range(h):
    for j in range(w):
        phi_vals = []
        for k in range(num_circles):
          phi_temp = np.sqrt((x[i][j]-x_vals[k])**2 + (y[i][j]-y_vals[k])**2)- radii[k]
          phi_vals.append(phi_temp)
          phi[i,j] = min(phi_vals)
        if phi[i,j] > 0:
          norm_factor += 1
  return phi, norm_factor

def initial_point(h, w, phi):
  # SET x0 AND FIND VISIBILITY FOR x0; choose initial point x0 that is not in a black region
  x_init = random.randint(1, w-2)
  y_init = random.randint(1, h-2)

  in_obstacle = True
  while in_obstacle:
    if phi[x_init, y_init] <= 0:
      x_init = random.randint(1, w-2)
      y_init = random.randint(1, h-2)
    else:
      in_obstacle = False
      break

  x0 = np.array([[x_init, y_init]])
  return x0

def calc_frontiers(h, w, phi, psi_cum, dx):
  return delta(psi_cum,2*dx)*dx*(delta(phi,2*dx)==0)

def compute_gain(phi, psi_cum, x0, gain_tol, norm_factor, h, w, finished):
  # Index with highest gain is the next vantage point
  gain_max = 0 # To keep track of maximum gain across all pixels
  gain_x0 = np.zeros(shape=(h,w), dtype=float) # Gain is h*w matrix with gain for each potential new vantage pt
  psi_temp = np.ndarray(shape=(h,w), dtype=float) # This will contain total visibility for each pixel

  for i in range (h):
    for j in range (w): # How much you can see at each point--max of this should be next point
      if phi[i,j]>0: # Only do computation for points not in the obstacles
        psi_temp = np.array(vis2d(phi, np.array([i,j]))) # Total visibility of point x_i

        # Compute gain function at potential vantage point z
        for k in range (h):
          for l in range (w):
            '''
            psi_cum[k,l] < 0 means we couldn't see this point before from our collection of vantage points
            psi_temp[k,l] > 0 means we can currently see this point from candidate vantage point [i,j]
            phi[k,l] > 0 means it's not inside an obstacle
            '''
            if (psi_cum[k,l]<0) and (psi_temp[k,l]>0) and (phi[k,l]>0): # <=0 includes boundaries
              gain_x0[i,j] += 1 # The gain is one more grid square/pixel

  # Find the point with maximum gain to be the next candidate point
  x_max = -1
  y_max = -1
  for i in range (h):
    for j in range (w):
      if (gain_x0[i,j] > gain_max) and ([i,j] not in x0):
        gain_max = gain_x0[i,j]
        x_max = i
        y_max = j
  if (gain_max/norm_factor)<gain_tol: # Stop when gains are no longer significant
    print('no more gains')
    finished = True
  x1 = np.array([[x_max, y_max]])
  x0 = np.append(x0, x1, axis=0) # This is an array containing all vantage points
  print('gain max:', gain_max)
  print('the chosen vantage point is: (' + str(x_max) + ',' + str(y_max) +')')

  gain_x0 = gain_x0/norm_factor
  return gain_x0, x0, finished

def greedy_alg(psi_cum, phi, psi_0, x0, frontiers, h, w, iteration_counter, norm_factor):
  # FIND THE OTHER POINTS/FINISH THE GREEDY ALGORITHM
  base_dir = '/content/drive/MyDrive/Colab_Notebooks'
  counter = 0 # Counter is used to save the cumulative visibility and frontiers and gains under the same number/label
  finished = False
  dx = 2/h
  gain_tol = 1e-5
  psi_1 = psi_0

  while not finished:
    # Save the cumulative visibility and frontiers
    #torch.save(psi_cum, base_dir + "/cumulative_visibility/cumulative_visibility_{}_{}.pt".format(iteration_counter, counter))
    #torch.save(frontiers, base_dir + "/frontiers/frontiers_{}_{}.pt".format(iteration_counter, counter))

    # Compute gain function and the next vantage point
    gain_x0, x0, finished = compute_gain(phi, psi_cum, x0, gain_tol, norm_factor, h, w, finished)
    plt.imshow(gain_x0); plt.show(); plt.close()
    #torch.save(gain_x0, base_dir + "/gains/gain_{}_{}.pt".format(iteration_counter, counter))

    # PLOT THE POINTS WITH CURRENT VANTAGE POINT
    h_1, m_1= x0.shape
    psi_1 = np.array(vis2d(phi, x0[-1])) # Compute new visibility
    print('this is x{}'.format(counter))
    #plot_pos(psi_1, x0_one = (x0[0])[1], x0_two = (x0[0])[0], h_1 = h_1, x0 = x0)

    # Calculate new cumulative visibility
    psi_1 = distance(psi_1,dx)
    psi_cum = np.maximum(psi_cum, psi_1)
    frontiers = calc_frontiers(h, w, phi, psi_cum, dx) # Compute new frontiers

    # Plot cumulative visibility and frontiers
    plot_pos(psi_cum, x0_one = (x0[0])[1], x0_two = (x0[0])[0], h_1 = h_1, x0 = x0)
    plot_pos(frontiers)

    # PLOT THE FRONTIERS (INTERSECTIONS OF WHAT CAN BE SEEN AND WHAT COULD BE SEEN)
    # plt.plot()
    counter += 1


def run_sequence(iteration_counter, output_path = None):
# demonstration of how to setup the scene and observing locations to use the visibility algorithm to find the path 
# psi is visibility

  device=torch.device("cuda")
  # SET UP VARIABLES
  h,w = 64,64
  m = h
  dx = 2/m 
  tol = 1e-2
  eps = 2*dx

  # Calculate initial cumulative visibility from initial point
  psi_cum = -100+np.zeros(shape=(h,w), dtype=float) # Cumulative visibility
  phi, norm_factor = setup(h, w)
  x0 = initial_point(h, w, phi)

  psi_0 = np.array(vis2d(phi, x0[0])) # This is the visibility from x0
  psi_0 = distance(psi_0,dx) # distance is a function from skfmm (sci kit fast marching method)
  psi_cum = np.maximum(psi_cum, psi_0) 
  
  frontiers = calc_frontiers(h, w, phi, psi_cum, dx)

  # PLOT PSI_0 AND PSI_CUM AND FRONTIERS
  print('the environment is')
  plot_pos(phi)
  print('this is x0')
  print('psi_0:')
  plot_pos(psi_0, (x0[0])[1], (x0[0])[0])
  print('initial cumulative visibility')
  plot_pos(psi_cum, (x0[0])[1], (x0[0])[0])
  print('frontiers')
  plot_pos(frontiers)

  '''
  iteration_counter is the labeled number of the environment
  example: iteration_counter = 3000 means this is the 3000th map
  '''
  greedy_alg(psi_cum, phi, psi_0, x0, frontiers, h, w, iteration_counter, norm_factor)